In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import pickle

from datetime import datetime, date, time
from textblob import TextBlob
from textblob import Word
from pre_processing import *
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Andres
[nltk_data]     Montero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]


In [32]:
DATA_FOLDER = './data'

tweets_col_names=['text']

tweets_dtypes = {'text': str }

cleaned_pos = True
cleaned_neg = True

In [4]:
tweets_neg = pd.read_fwf(DATA_FOLDER + '/train_neg.txt',  names=tweets_col_names,
                              dtype=tweets_dtypes)
tweets_neg.head(20)

,text
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,glad i dot have taks tomorrow ! ! #thankful #s...
2,1-3 vs celtics in the regular season = were fu...
3,<user> i could actually kill that girl i'm so ...
4,<user> <user> <user> i find that very hard to ...
5,wish i could be out all night tonight ! <user>
6,<user> i got kicked out the wgm
7,rt <user> <user> <user> yes she is ! u tell it...
8,why is she so perfect <url>
9,<user> hi harry ! did u havea good time in aus...


In [5]:
tweets_neg.drop_duplicates(inplace=True)

In [6]:
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: filter_digits(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: separate_hash(x))
tweets_neg['text'] = tweets_neg['text'].str.replace('#', '')
tweets_neg['text'] = tweets_neg['text'].str.replace('ur', 'your')
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: expand_contractions(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: interpret_emoji(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_punctuation(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_words(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: replace_moreletters(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: split_number_text(x))

tweets_neg.head()

,text
0,vinco tresorpack difficulty of object disassem...
1,glad i dot have taks tomorrow thankful startho
2,vs celtics in the regular season were fucked i...
3,i could actually kill that girl i am so sorry
4,i find that very hard to believe im afraid


In [7]:
stop = stopwords.words('english')
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
tweets_neg['text'].head()

0    vinco tresorpack difficulty object disassemble...
1              glad dot taks tomorrow thankful startho
2       vs celtics regular season fucked play playoffs
3                       could actually kill girl sorry
4                          find hard believe im afraid
Name: text, dtype: object

### Remove more comon words of the data set

In [8]:
freq_max = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[:10]
freq_max

yourl     28483
frame      6690
rt         4225
like       4093
get        3756
yoyour     3698
go         3662
one        3336
want       3259
miss       3110
dtype: int64

In [9]:
freq_max = list(freq_max.index)
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max))
tweets_neg['text'].head()

0    vinco tresorpack difficulty object disassemble...
1              glad dot taks tomorrow thankful startho
2       vs celtics regular season fucked play playoffs
3                       could actually kill girl sorry
4                          find hard believe im afraid
Name: text, dtype: object

In [10]:
freq_min = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[-10:]
freq_min

kinesis          1
hoa              1
bally            1
sasn             1
triggablueivy    1
imaloser         1
thismornin       1
orman's          1
quiver           1
fotopoulos       1
dtype: int64

In [11]:
freq_min = list(freq_min.index)
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min))
tweets_neg['text'].head()

0    vinco tresorpack difficulty object disassemble...
1              glad dot taks tomorrow thankful startho
2       vs celtics regular season fucked play playoffs
3                       could actually kill girl sorry
4                          find hard believe im afraid
Name: text, dtype: object

### Correct spelling, takes a long time ! for now we will coment it and use it later

In [12]:
# tweets_neg['text'] = tweets_neg['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
# tweets_neg

### Tokenization

In [13]:
# tweets_neg['text'] = tweets_neg['text'].apply(lambda x: (TextBlob(x).words))

In [14]:
# tweets_neg.head()

### Lematization

In [15]:
tweets_neg['text_lema'] = tweets_neg['text'].apply( lemmatize_text)
tweets_neg

,text,text_lema
0,vinco tresorpack difficulty object disassemble...,"[vinco, tresorpack, difficulty, object, disass..."
1,glad dot taks tomorrow thankful startho,"[glad, dot, taks, tomorrow, thankful, startho]"
2,vs celtics regular season fucked play playoffs,"[v, celtic, regular, season, fucked, play, pla..."
3,could actually kill girl sorry,"[could, actually, kill, girl, sorry]"
4,find hard believe im afraid,"[find, hard, believe, im, afraid]"
5,wish could night tonight,"[wish, could, night, tonight]"
6,got kicked wgm,"[got, kicked, wgm]"
7,yes tell lips closed okay,"[yes, tell, lip, closed, okay]"
8,perfect,[perfect]
9,hi harry havea good time aus didnt see maybe n...,"[hi, harry, havea, good, time, au, didnt, see,..."


## Save pickle

In [19]:
if not cleaned_neg :
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_neg.pickle","wb")
    pickle.dump(tweets_neg['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
else:
    print("Already Trained!")

Already Trained!


In [20]:
pickle_in = open("data/pre_processed/tweets_neg.pickle","rb")
test_pickle = pickle.load(pickle_in)
test_pickle

0        [into, tresorpack, difficulty, object, disasse...
1           [glad, dot, take, tomorrow, thankful, started]
2        [v, celtic, regular, season, tucked, play, pla...
3                     [could, actually, kill, girl, sorry]
4                        [find, hard, believe, in, afraid]
5                            [wish, could, night, tonight]
6                                        [got, kicked, wm]
7                           [yes, tell, lip, closed, okay]
8                                                [perfect]
9        [hi, harry, have, good, time, a, didn, see, ma...
10       [introduction, programming, +, +, nd, edition,...
13                                              [white, a]
14                      [dan, love, sad, wheresthegeneral]
15                  [many, wonderful, building, do, still]
16                        [annoying, secretly, find, good]
17       [post, boom, spanish, american, fiction, sun, ...
18       [layer, heart, paperback, journey, inspired, r.

In [21]:
tweets_pos = pd.read_fwf(DATA_FOLDER + '/train_pos.txt',  names=tweets_col_names,
                              dtype=tweets_dtypes)
tweets_pos.drop_duplicates(inplace=True)
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: filter_digits(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: separate_hash(x))
tweets_pos['text'] = tweets_pos['text'].str.replace('#', '')
tweets_pos['text'] = tweets_pos['text'].str.replace('ur', 'your')
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: expand_contractions(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: interpret_emoji(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_punctuation(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_words(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: replace_moreletters(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: split_number_text(x))

In [22]:
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [23]:
freq_max_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[:10]
freq_max_pos = list(freq_max_pos.index)
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_pos))

In [24]:
freq_min_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[-10:]
freq_min_pos = list(freq_min_pos.index)
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_pos))

In [25]:
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))

In [26]:
tweets_pos['text_lema'] = tweets_pos['text'].apply( lemmatize_text)
tweets_pos

,text,text_lema
0,funny austin read mention austin god knows hop...,"[funny, austin, read, mention, austin, god, kn..."
1,logic dumb even crop name photo ask,"[logic, dumb, even, crop, name, photo, ask]"
2,put caper box looked battle crakkbitch,"[put, caper, box, looked, battle, crakkbitch]"
3,thanks sir trip ll mamma keep down a than,"[thanks, sir, trip, ll, mamma, keep, down, a, ..."
4,visiting brother mr beset birthday gift every,"[visiting, brother, mr, beset, birthday, gift,..."
5,may lifecompleted sweet facebook let please,"[may, lifecompleted, sweet, facebook, let, ple..."
6,dnextalbumtitle feel rollercoaster life song c...,"[dnextalbumtitle, feel, rollercoaster, life, s..."
7,working hard hardly working harder's future wo...,"[working, hard, hardly, working, harder's, fut..."
8,saw replying bit,"[saw, replying, bit]"
9,belong,[belong]


In [30]:
if not cleaned_pos:  
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_pos.pickle","wb")
    pickle.dump(tweets_pos['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
else:
    print("Already Trained!")

Saving file with preprocessed Tweets
Saved!


In [31]:
pickle_in = open("data/pre_processed/tweets_pos.pickle","rb")
test_pickle_pos = pickle.load(pickle_in)
test_pickle_pos

0        [funny, austin, read, mention, austin, god, kn...
1              [logic, dumb, even, crop, name, photo, ask]
2            [put, caper, box, looked, battle, crakkbitch]
3        [thanks, sir, trip, ll, mamma, keep, down, a, ...
4        [visiting, brother, mr, beset, birthday, gift,...
5        [may, lifecompleted, sweet, facebook, let, ple...
6        [dnextalbumtitle, feel, rollercoaster, life, s...
7        [working, hard, hardly, working, harder's, fut...
8                                     [saw, replying, bit]
9                                                 [belong]
10                                  [and, cheer, national]
11       [send, invitation, shop, line, find, everythin...
12                                [woke, finn, go, church]
13                                [agreed, day, left, the]
14                                       [money, katemelo]
15                   [dam, alexis, got, lot, say, twitter]
16       [grateful, today, dream, fulfilled, heart, ful.